# Sentiment Analysis on Tweets for Stocks

In [17]:
# Libraries
import twint
import nest_asyncio
nest_asyncio.apply()
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import datetime, timedelta

# Gathering Tweets

## Function for Twint

In [18]:
def getTweets(search_term, date, limit):
    """
    Configures Twint and returns a dataframe of the results.
    """
    # Configuring Twint for search
    c = twint.Config()

    # The limit of tweets to retrieve
    c.Limit = limit

    # Count of the number of tweets fetched
    c.Count = True

    # Search term
    c.Search = search_term

    # Removing retweets
    c.Filter_retweets = True

    # Getting popular tweets
    c.Popular_tweets = True

    # Lowercasing tweets
    c.Lowercase = True

    # English only
    c.Lang ='en'

    # Excluding tweets with links
    c.Links ='exclude'

    # Tweets until a specified date
    c.Until = date

    # Making the results pandas friendly
    c.Pandas = True
    
    # Stopping print in terminal
    c.Hide_output = True

    # Searching
    twint.run.Search(c)

    # Assigning the DF
    df = twint.storage.panda.Tweets_df
    
    # Filtering out other cashtag mentions
    df = df[df['cashtags'].apply(lambda x: len(x)==1)]
    
    return df

## Using Twint
Tweet retrieval over a period of time.

In [19]:
def tweetByDay(start, stop, df):
    """
    Gathers a set amount of tweets daily between two dates and returns a dataframe.
    """
    # Finishing the recursive loop
    if start==stop:
        return df
    
    # Appending the new set of tweets for the day
    df = df.append(getTweets("$BBIG", start, 20), ignore_index=True)
    
    # The new start date
    new_start = (datetime.strptime(start, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d")
    
    print(new_start)
    
    return tweetByDay(new_start, stop, df)

In [20]:
# Getting tweets daily
df = tweetByDay("2021-09-01", "2021-08-29", pd.DataFrame())

[+] Finished: Successfully collected 20 Tweets.
2021-08-31
[+] Finished: Successfully collected 20 Tweets.
2021-08-30
[+] Finished: Successfully collected 20 Tweets.
2021-08-29


In [21]:
df

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1432855658156670976,1432855658156670976,1.630454e+12,2021-08-31 19:59:44,-0400,,I keep adding $BBIG thank you shorts lol,en,[],[bbig],...,,,,,,[],,,,
1,1432855436642770945,1432854897330847748,1.630454e+12,2021-08-31 19:58:51,-0400,,@sanjose_408 I hope so too! We have tons of me...,en,[],[bbig],...,,,,,,"[{'screen_name': 'sanjose_408', 'name': 'marko...",,,,
2,1432855344607285248,1432855344607285248,1.630454e+12,2021-08-31 19:58:29,-0400,,$BBIG 3 minutes to cash out and get the BANK ...,en,[],[bbig],...,,,,,,[],,,,
3,1432855334222286849,1432805391709179907,1.630454e+12,2021-08-31 19:58:27,-0400,,@MarketRebels We need Aug 30th IV for 9-1 $BBIG,en,[],[bbig],...,,,,,,"[{'screen_name': 'MarketRebels', 'name': 'Mark...",,,,
4,1432855214709637128,1432855214709637128,1.630454e+12,2021-08-31 19:57:59,-0400,,$BBIG watch it pump last two minutes,en,[],[bbig],...,,,,,,[],,,,
5,1432855147642716162,1432855147642716162,1.630454e+12,2021-08-31 19:57:43,-0400,,"Who ever try’s to short $BBIG , your moms a ho!",en,[],[bbig],...,,,,,,[],,,,
6,1432854764677701633,1432820797442301952,1.630454e+12,2021-08-31 19:56:11,-0400,,@DrMagoops No mfer shorting $bbig is my friend,en,[],[bbig],...,,,,,,"[{'screen_name': 'DrMagoops', 'name': 'Dr. Mag...",,,,
7,1432854728896090114,1432854728896090114,1.630454e+12,2021-08-31 19:56:03,-0400,,$bbig Drop that proxy at 8 and it's game over ...,en,[],[bbig],...,,,,,,[],,,,
8,1432854387567706116,1432804287713742849,1.630454e+12,2021-08-31 19:54:41,-0400,,@avivhadar One of us is right. Let the battle ...,en,[],[bbig],...,,,,,,"[{'screen_name': 'avivhadar', 'name': 'Aviv Ha...",,,,
9,1432854181262479366,1432854181262479366,1.630454e+12,2021-08-31 19:53:52,-0400,,$BBIG make sure you read through most recent ...,en,[],[bbig],...,,,,,,[],,,,
